In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! unzip /content/drive/MyDrive/3d-machine-learning/shapenet_prepared.h5.zip -d  /content/

Archive:  /content/drive/MyDrive/3d-machine-learning/shapenet_prepared.h5.zip
  inflating: /content/shapenet_prepared.h5  


In [ ]:
!python train_UNet.py

Using device: cuda:0
[000/00000] train_loss: 3.933707
[001/00000] train_loss: 3.931664
[002/00000] train_loss: 3.929264
[003/00000] train_loss: 3.926023
[004/00000] train_loss: 3.921690
[005/00000] train_loss: 3.916083
[006/00000] train_loss: 3.908959
[007/00000] train_loss: 3.900022
[008/00000] train_loss: 3.888931
[009/00000] train_loss: 3.875306
[010/00000] train_loss: 3.858685
[011/00000] train_loss: 3.838520
[012/00000] train_loss: 3.814269
[013/00000] train_loss: 3.785334
[014/00000] train_loss: 3.751064
[015/00000] train_loss: 3.710809
[016/00000] train_loss: 3.662699
[017/00000] train_loss: 3.607651
[018/00000] train_loss: 3.544725
[019/00000] train_loss: 3.474028
[019/00000] val_loss: 3.396160 | best_val_loss: 3.396160
[020/00000] train_loss: 3.396160
[021/00000] train_loss: 3.312090
[022/00000] train_loss: 3.223574
[023/00000] train_loss: 3.133108
[024/00000] train_loss: 3.043346
[025/00000] train_loss: 2.956525
[026/00000] train_loss: 2.873831
[027/00000] train_loss: 2.79523

In [ ]:
#FOR OVERFIT #######################################################################
####################################################################################
####################################################################################
import torch
import numpy as np
from dataset import ShapeNetPartDataset
from model import MultiSacleUNet
import h5py

class_name = np.genfromtxt('all_object_categories.txt',dtype='U')[:,0]

dataset_path = "shapenet_prepared.h5"
f = h5py.File(dataset_path,'r')
x_test = f['x_val'][0]
y_test = f['y_val'][0]
s_test = f['s_val'][0]
p_test = f['p_val'][0]

# give the location of your pre-trained model
ckpt = "/content/runs/temp_model_1/model.ckpt"
model = MultiSacleUNet()
model.load_state_dict(torch.load(ckpt, map_location='cpu'))
model.eval()

#holds the class specific labels, will 12,13,14 in overfit case
class_label_region = np.zeros((1,2),dtype=np.int64)

label_min = s_test.min()
label_max = s_test.max()

# this double indexing is from the test script for multiple examples, not necessary to change since the rest works according to this
class_label_region[0,0] = label_min
class_label_region[0,1] = label_max

# testing for multiple examples normally reads from the h5 files in these lines but it's not necessary for a single example
x_set = x_test # point cloud position in 3D
y_set = y_test # point cloud shape class
s_set = s_test # point cloud segments
p_set = p_test # 2D position
pre_set = np.zeros((1,2048,1))

In [ ]:
#need to have shapenet_prepared.h5 file ready under same directory with this file
test_dataset = ShapeNetPartDataset(
    path='shapenet_prepared.h5',
    split='overfit'
    )
pre_test = np.zeros_like(s_test)
pre_test.shape

(2048,)

In [ ]:
pre_sample = np.zeros_like(pre_test)
print(pre_sample.shape)
#getting class specific min and max labels, it's actually same with the lines 149-150 in overfit case. 
#I'll change it once we figure out proper training
label_min = int(class_label_region[0,0])
label_max = int(class_label_region[0,1]+1)

label_min, label_max

(2048,)


(0, 4)

In [ ]:
test_dataloader = torch.utils.data.DataLoader(
    test_dataset,  
    batch_size=1,
)


In [ ]:
for idx,i in enumerate(test_dataloader):
  input_tensor = i["3d_points"]
  input_label = i["part_label"]

In [ ]:
input_tensor.shape, input_label.shape

(torch.Size([1, 3, 256, 256]), torch.Size([1, 256, 256]))

In [ ]:
pre_image = model(input_tensor)
pre_image.shape

torch.Size([1, 51, 256, 256])

In [ ]:
torch.nn.CrossEntropyLoss()(pre_image, input_label)

tensor(0.3394, grad_fn=<NllLoss2DBackward0>)

In [ ]:
pre_image[:,label_min:label_max,:,:].argmax(1)


tensor([[[3, 2, 1,  ..., 3, 2, 3],
         [2, 1, 0,  ..., 2, 3, 2],
         [1, 0, 3,  ..., 3, 2, 1],
         ...,
         [3, 2, 3,  ..., 0, 3, 3],
         [2, 3, 2,  ..., 3, 3, 2],
         [3, 2, 1,  ..., 3, 2, 3]]])

In [ ]:
pre_image = pre_image[:,:,:,:].argmax(1)+label_min
pre_image.shape

torch.Size([1, 256, 256])

In [ ]:
pre_image, input_label

(tensor([[[13, 50, 50,  ..., 50, 50, 13],
          [50, 50, 50,  ..., 50, 13, 50],
          [50, 50, 50,  ..., 13, 50, 50],
          ...,
          [50, 50, 49,  ..., 50, 50, 50],
          [50, 49, 50,  ..., 50, 50, 50],
          [49, 50, 50,  ..., 50, 50, 49]]]),
 tensor([[[50, 50, 50,  ..., 50, 50, 50],
          [50, 50, 50,  ..., 50, 50, 50],
          [50, 50, 50,  ..., 50, 50, 50],
          ...,
          [50, 50, 50,  ..., 50, 50, 50],
          [50, 50, 50,  ..., 50, 50, 50],
          [50, 50, 50,  ..., 50, 50, 50]]]))

In [ ]:
np.bincount(pre_image.flatten())

array([  456,   319,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,   262,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,   252, 64247])

In [ ]:
np.bincount(input_label.flatten())


array([  952,   648,   257,   191,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0, 63488])

In [ ]:
pre_sample = pre_image[:,p_test[:,0],p_test[:,1]]
pre_sample = pre_sample[0]
pre_sample.shape

torch.Size([2048])

In [ ]:
pre_sample

tensor([50, 50, 50,  ..., 50, 50, 50])

In [ ]:
pre_test = pre_sample

In [ ]:
pre_test

tensor([50, 50, 50,  ..., 50, 50, 50])

In [ ]:
#iou calculation, will only calculate one class which is chair for overfit example
iou_shape = np.zeros(len(s_test))
#for idx_sample,pos,obj_class in zip(range(0,len(p_test)),p_test,y_test):

label_min = int(class_label_region[0,0])
label_max = int(class_label_region[0,1]+1)
iou_list = []
_pre = pre_test
gt_sample = s_test
# % for each segment, calculate iou
for i_class in range(0,51):
    # break
    if len(np.where(_pre == i_class)[0]) == 0:
      continue
    print("class:",i_class,"number of points:",len(np.where(_pre == i_class)[0]))
    tp = torch.sum( (_pre == i_class) * (gt_sample == i_class) )
    fp = torch.sum( (_pre == i_class) * (gt_sample != i_class) )
    fn = torch.sum( (_pre != i_class) * (gt_sample == i_class) )

    # % if current segment exists then count the iou
    iou = (tp+1e-12) / (tp+fp+fn+1e-12)

    iou_list.append(iou)

iou_shape[0] = np.mean(iou_list)
iou_list

class: 0 number of points: 11
class: 1 number of points: 10
class: 9 number of points: 3
class: 50 number of points: 2024


[tensor(0.0042), tensor(0.0077), tensor(3.3333e-13), tensor(4.9407e-16)]

In [ ]:
print( 'iou_instance =', iou_shape[0].mean())


iou_instance = 0.0029569948092103004


In [ ]:
# %%
#VISUALIZATION
#visualizatin normally will be a different script by itself but just for this overfit sample visualization
#the code is copied to here so some repetition of the code might be seen, not a problem, will be fixed
# install plotly first then it might give an error while plotting, something about "nb", only need to update some package then restart, it fixed for me
import numpy as np
import seaborn as sns
import plotly.graph_objects as go

current_palette = sns.color_palette('bright',10)
#"all_object_categories.txt" should also be under the same directory with this script
class_name = np.genfromtxt('all_object_categories.txt',dtype='U')[:,0]

In [ ]:
label_min = s_test.min()
label_max = s_test.max()
x_pt = x_set
s_pt = s_set-label_min
pre_pt = pre_sample-label_min

In [ ]:
pre_pt

tensor([50, 50, 50,  ..., 50, 50, 50])

In [ ]:
for i_seg in range(1,51):
    idxs = np.where(pre_pt == i_seg)[0]
    print(len(idxs))


10
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2024


In [ ]:
#for prediction
fig = go.Figure()
k = -1
for i_seg in range(50):
    idxs = np.where(pre_pt == i_seg)[0]
    if len(idxs) == 0:
      continue
    else:
      k += 1
    color = current_palette.as_hex()[k]
    fig.add_trace(go.Scatter3d(x=x_pt[idxs,0], y=x_pt[idxs,1], z=x_pt[idxs,2],
                                   mode='markers',
                                   marker = dict(color = color )))
    
fig.show()

In [ ]:
#for ground truth
fig = go.Figure()
for i_seg in range(label_max - label_min +1):
    idxs = np.where(s_pt == i_seg)[0]
    color = current_palette.as_hex()[i_seg]
    fig.add_trace(go.Scatter3d(x=x_pt[idxs,0], y=x_pt[idxs,1], z=x_pt[idxs,2],
                                   mode='markers',
                                   marker = dict(color = color )))
    
fig.show()

In [ ]:
pre_pt.shape

torch.Size([2048])

In [ ]:
s_pt

array([1, 2, 1, ..., 1, 1, 0], dtype=int32)

In [ ]:
pre_pt

tensor([25, 31, 11,  ...,  7, 36, 46])